<a href="https://colab.research.google.com/github/shravanivadthya/E-commerce/blob/main/calculating_loyality_points.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

In [ ]:


gameplay = pd.read_excel("Analytics Position Case Study.xlsx", sheet_name="User Gameplay data")
deposit = pd.read_excel("Analytics Position Case Study.xlsx", sheet_name="Deposit Data")
withdrawal = pd.read_excel("Analytics Position Case Study.xlsx", sheet_name="Withdrawal Data")

In [ ]:
gameplay['Game_Play_Time'] = pd.to_datetime(gameplay['Game_Play_Time'])
deposit['Deposit_Time'] = pd.to_datetime(deposit['Deposit_Time'])
withdrawal['Withdrawal_Time'] = pd.to_datetime(withdrawal['Withdrawal_Time'])


In [ ]:
def calculate_loyalty_points(deposit_amt, withdrawal_amt, num_deposits, num_withdrawals, games_played):
    return (
        0.01 * deposit_amt +
        0.005 * withdrawal_amt +
        0.001 * max(num_deposits - num_withdrawals, 0) +
        0.2 * games_played
    )

In [ ]:
def get_slot_bounds(date, slot):
    if slot == "S1":
        start = datetime.combine(date, datetime.min.time())
        end = start + timedelta(hours=12)
    elif slot == "S2":
        start = datetime.combine(date, datetime.min.time()) + timedelta(hours=12)
        end = start + timedelta(hours=12)
    return start, end

In [ ]:
def compute_loyalty_for_slot(date_str, slot):
    date = datetime.strptime(date_str, "%d %B %Y")
    start, end = get_slot_bounds(date, slot)

In [ ]:
games = gameplay[(gameplay['Game_Play_Time'] >= start) & (gameplay['Game_Play_Time'] < end)]
    deposits = deposit[(deposit['Deposit_Time'] >= start) & (deposit['Deposit_Time'] < end)]
    withdrawals = withdrawal[(withdrawal['Withdrawal_Time'] >= start) & (withdrawal['Withdrawal_Time'] < end)]


In [ ]:
games_grouped = games.groupby('Player_ID').size().reset_index(name='Games_Played')
    deposits_grouped = deposits.groupby('Player_ID').agg({'Amount': 'sum', 'Deposit_Time': 'count'}).rename(columns={'Amount': 'Deposit_Amount', 'Deposit_Time': 'Num_Deposits'})
    withdrawals_grouped = withdrawals.groupby('Player_ID').agg({'Amount': 'sum', 'Withdrawal_Time': 'count'}).rename(columns={'Amount': 'Withdrawal_Amount', 'Withdrawal_Time': 'Num_Withdrawals'})


In [ ]:
    merged = pd.merge(games_grouped, deposits_grouped, on='Player_ID', how='outer')
    merged = pd.merge(merged, withdrawals_grouped, on='Player_ID', how='outer')
    merged.fillna(0, inplace=True)

In [ ]:
merged['Loyalty_Points'] = merged.apply(lambda row: calculate_loyalty_points(
        row['Deposit_Amount'], row['Withdrawal_Amount'],
        row['Num_Deposits'], row['Num_Withdrawals'],
        row['Games_Played']), axis=1)

    return merged[['Player_ID', 'Loyalty_Points']].sort_values(by='Loyalty_Points', ascending=False)


In [ ]:
slots_to_check = [
    ("2 October 2021", "S1"),
    ("16 October 2021", "S2"),
    ("18 October 2021", "S1"),
    ("26 October 2021", "S2"),
]
results = {}
for date_str, slot in slots_to_check:
    results[f"{date_str} {slot}"] = compute_loyalty_for_slot(date_str, slot)


In [ ]:
for key, df in results.items():
    print(f"\nLoyalty Points for {key}")
    print(df.head())


In [ ]:
oct_start = datetime(2021, 10, 1)
oct_end = datetime(2021, 11, 1)


In [ ]:
games_oct = gameplay[(gameplay['Game_Play_Time'] >= oct_start) & (gameplay['Game_Play_Time'] < oct_end)]
deposits_oct = deposit[(deposit['Deposit_Time'] >= oct_start) & (deposit['Deposit_Time'] < oct_end)]
withdrawals_oct = withdrawal[(withdrawal['Withdrawal_Time'] >= oct_start) & (withdrawal['Withdrawal_Time'] < oct_end)]


In [ ]:
games_grouped = games_oct.groupby('Player_ID').size().reset_index(name='Games_Played')
deposits_grouped = deposits_oct.groupby('Player_ID').agg({'Amount': 'sum', 'Deposit_Time': 'count'}).rename(columns={'Amount': 'Deposit_Amount', 'Deposit_Time': 'Num_Deposits'})
withdrawals_grouped = withdrawals_oct.groupby('Player_ID').agg({'Amount': 'sum', 'Withdrawal_Time': 'count'}).rename(columns={'Amount': 'Withdrawal_Amount', 'Withdrawal_Time': 'Num_Withdrawals'})


In [ ]:
merged = pd.merge(games_grouped, deposits_grouped, on='Player_ID', how='outer')
merged = pd.merge(merged, withdrawals_grouped, on='Player_ID', how='outer')
merged.fillna(0, inplace=True)

In [ ]:
merged['Loyalty_Points'] = merged.apply(lambda row: calculate_loyalty_points(
    row['Deposit_Amount'], row['Withdrawal_Amount'],
    row['Num_Deposits'], row['Num_Withdrawals'],
    row['Games_Played']), axis=1)

In [ ]:
merged.sort_values(by=['Loyalty_Points', 'Games_Played'], ascending=False, inplace=True)
merged.reset_index(drop=True, inplace=True)

In [ ]:
print("\nTop 50 Players for October:")
print(merged[['Player_ID', 'Loyalty_Points']].head(50))

In [ ]:
average_deposit_amount = deposits_oct['Amount'].mean()
average_deposit_per_user = deposits_oct.groupby('Player_ID')['Amount'].sum().mean()
average_games_per_user = games_oct.groupby('Player_ID').size().mean()

In [ ]:
print(f"\nAverage deposit amount: ₹{average_deposit_amount:.2f}")
print(f"Average total deposit per user in October: ₹{average_deposit_per_user:.2f}")
print(f"Average number of games per user in October: {average_games_per_user:.2f}")